In [ ]:
## getting data from the amazon ask questions section - https://www.amazon.com/ask/questions/asin/B0C6KC23JH/
## frame questions based on the attributes of the product

# Data Preparation for QnA model

## Installing and Importing Libraries

In [6]:
import os
import json
import re
import string
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Optionally, you can reset the warning filters later if needed
# warnings.resetwarnings()

In [7]:
data_folder = r"C:\\Users\\likhi\\Documents\\02 Pycharm Datasets\\01 Master Thesis\\04 Product Data\\"
faq_data_folder = r"C:\\Users\\likhi\\Documents\\02 Pycharm Datasets\\01 Master Thesis\\07 QnA\\QnA Limit 100\\"
destination_data_dir = r'C:\\Users\\likhi\\Documents\\02 Pycharm Datasets\\01 Master Thesis\\07 QnA\\training_test_data\\'

if not os.path.exists(destination_data_dir):
    os.mkdir(destination_data_dir)

## Getting the Product Data/ Saving Training Files

In [15]:
## find sentences and remove sentences with see more

def clean_descriptions(desc):
    #table = str.maketrans(' ', ' ', string.punctuation)
    desc = desc.split(' ')
    #desc = [word.lower() for word in desc]
    #desc = [w.translate(table) for w in desc]
    desc = [word for word in desc if len(word)>1]
    desc = ' '.join(desc)
    # pattern = r'[0-9]'
    # desc = re.sub(pattern, '', desc)   
    
    return desc


In [11]:
from transformers import T5Tokenizer

model_name =  "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
# Function to process a single product
def process_product(product_folder, category_path, product_faq_data):
    
    product_path = os.path.join(category_path, product_folder)
   
    try:
        data = json.load(open(os.path.join(product_path, product_folder + ".json"), "r", encoding="utf-8"))
        product_overview = data.get('product_overview', '')
        product_overview_text = "Product overview is " + " ".join([key + " is " + value 
                                                            for key, value in product_overview.items() 
                                                            ])
        

        product_description = "Product description is " +",".join(data.get('description', ''))
        # categories = ",".join(data.get('categories', ''))
        product_title = "Product title is " + data.get('Title', '')
        final_description = product_title + ". " + product_overview_text + ". " + product_description
    
        cleaned_final_description = clean_descriptions(final_description)
        
        product_overview_questions = ["What is the " + key + " of the product?" 
                                      for key in product_overview.keys()]
        
        product_overview_answers = [value + " is the " + key +" of the product." 
                                    for key, value in product_overview.items()]

        if not product_faq_data != list():
            faq_questions = [i[0] for i in product_faq_data]
            faq_answers = [i[1] for i in product_faq_data]
        else:
            faq_questions = []
            faq_answers = []
            
        total_questions = product_overview_questions + faq_questions
        total_answers = product_overview_answers + faq_answers
        
        # 
        # encoding = tokenizer.encode(cleaned_final_description, return_tensors="pt",
        #                             max_length=2048, pad_to_max_length=True,
        #                             truncation=True)
        # 
        # encoding_shape = encoding.shape[1]
        
        for question, answer in zip(total_questions, total_answers):
            context = cleaned_final_description
            context_file.write(context+'\n')
            training_question = question
            questions_file.write(training_question + '\n')
            training_answer = answer
            answers_file.write(training_answer + '\n')
            
            context_and_question_length.append(len("context: " + context + ". Question: " + training_question))
            
    except Exception as e:
        pass

        # print(f"Error processing product {product_folder}: {e}")
        # print(f"Path to product folder: {product_path}")
        # print(f"Content of product folder: {os.listdir(product_path)}")


# Initialize lists to keep track of product names
all_product_names = []

# Tracking the max length of context and question
context_and_question_length = []

# # Open separate output files for train, validation, and test data
context_file = open(os.path.join(destination_data_dir, 'context_file.txt'), 'w', encoding='utf-8')
questions_file = open(os.path.join(destination_data_dir, 'questions_file.txt'), 'w', encoding='utf')
answers_file = open(os.path.join(destination_data_dir, 'answers_file.txt'), 'w', encoding='utf-8')


for category_folder in tqdm(sorted(os.listdir(data_folder))):
    product_category_path = os.path.join(data_folder, category_folder)
    qna_category_path = os.path.join(faq_data_folder, category_folder + "_Amazon QnA_data.json")
    
    category_faq_data = json.load(open(qna_category_path, 'r', encoding='utf-8'))
    
    for product_folder in sorted(os.listdir(product_category_path)):
        
        if product_folder not in all_product_names:                     
            all_product_names.append(product_folder)
            
            try:
                product_faq_data = category_faq_data[product_folder]
            except:
                product_faq_data = list()
            
            # Process each product using multiprocessing pool
            process_product(product_folder, product_category_path, product_faq_data)
# 
# # Close the output files
context_file.close()
questions_file.close()
answers_file.close()

print("Output files created successfully.")

100%|██████████| 265/265 [09:09<00:00,  2.08s/it]

Output files created successfully.


In [42]:
max(context_and_question_length)

12610

## Loading the text files

In [8]:
contexts_text = open(os.path.join(destination_data_dir, 'context_file.txt'), 'r', encoding='utf-8').read()
answers_text = open(os.path.join(destination_data_dir, 'answers_file.txt'), 'r', encoding='utf-8').read()
questions_text = open(os.path.join(destination_data_dir, 'questions_file.txt'), 'r', encoding='utf-8').read()

contexts = contexts_text.split('\n')
answers = answers_text.split('\n')
questions = questions_text.split('\n') 

In [9]:
print(contexts[:1], answers[:1], questions[:1])

['Product title is Perler Beads Assorted Multicolor Fuse Beads for Kids Crafts, 11000 pcs. Product overview is Color is Multicolor Material is Plastic Size is 11,000 Count Brand is Perler Shape is Round Item Weight is 1.69 Pounds Number of Pieces is 11000. Product description is Includes (11000) assorted Perler fuse beads and reusable ironing paper in plastic storage jar. This mega set of 11,000 Perler fuse beads comes with 30 different colors, including toothpaste, pastel lavender, butterscotch, and neon pink. Use your assorted Perler fuse beads with pre-made Perler bead design or get creative and make your own. These multicolor Perler beads are great arts and crafts activity for children. Use Perler pegboards, ironing paper, and an iron to complete your craft. Multicolor Perler beads set suitable for ages and up.'] ['Multicolor is the Color of the product.'] ['What is the Color of the product?']


In [16]:
len(contexts)

count =0 

for i in contexts:
    if "Product title is Product" in i:
        count+=1 

print(count)

20


## Creating PyTorch Dataset

In [48]:
from transformers import T5Tokenizer
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, contexts, questions, answers, tokenizer, max_length=15360):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.contexts)
    
    def __getitem__(self, idx):
        context = self.contexts[idx]
        question = self.questions[idx]
        answer = self.answers[idx]
        
        input_text = f"question: {question} context: {question}"
        target_text = answer
        
        # Tokenize input and target text
        input_ids = self.tokenizer.encode(input_text, return_tensors="pt", max_length=self.max_length, pad_to_max_length=True, truncation=True)
        target_ids = self.tokenizer.encode(target_text, return_tensors="pt", max_length=self.max_length, pad_to_max_length=True, truncation=True)
        
        # Ensure both input and target sequences have the same length
        max_seq_length = max(input_ids.size(1), target_ids.size(1))
        
        input_ids = input_ids[:, :max_seq_length]
        target_ids = target_ids[:, :max_seq_length]
                
        return {"input_ids": input_ids.squeeze(0), "target_ids": target_ids.squeeze(0)}


In [50]:
# Prepare custom dataset
custom_dataset = CustomDataset(contexts, questions, answers, tokenizer)
custom_dataset

### Creating train-test split

In [51]:
from torch.utils.data import random_split

train_size = int(0.8 * len(custom_dataset))  
test_size = len(custom_dataset) - train_size 

train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])

In [52]:
train_dataset

### Saving the dataset

In [54]:
import pickle

# Save custom dataset
with open(os.path.join(destination_data_dir, "QnA_train_dataset.pkl"), "wb") as f:
    pickle.dump(train_dataset, f)
    
# Save custom dataset
with open(os.path.join(destination_data_dir, "QnA_test_dataset.pkl"), "wb") as g:
    pickle.dump(test_dataset, g)

# # Load custom dataset
# with open("custom_dataset.pkl", "rb") as f:
#     loaded_custom_dataset = pickle.load(f)